## Problema
Muitos dos dados provenientes do JSON possuem erros. Além disso, "00_extracao_dados_full" gerou duplicações ao fazer a expansão de uma lista de "assuntos" e uma lista de "movimentos". De forma que os dados da forma como se apresentam agora não são adequados para o cálculo do indicador usado na solução.

Além disso, a maioria dos movimentos processuais são compostas por "trâmites" processuais, e não por "decisões" propriamente ditas, e que geram uma real mudança no estado do processo.

## Solução
Gerar uma visualização dos movimentos considerados relevantes para a solução proposta, onde:
- utiliza-se uma tabela [tabMovimentoRelevante.csv] onde foram mapeados quais movimentos são considerados aqui relevantes;
- faz-se a melhoria da qualidade dos dados necessária;
- criação de uma visualização de dados com foco nos movimentos relevantes, e remoção dos elementos que dizem respeito a entidade "processo" em si, pois essa já foi calculada em momento anterior.

In [11]:
import pandas as pd 
import numpy as np
from datetime import datetime

local_dados = 'dados/02_tabelas_transformadas/'

In [12]:
def obter_tabMovimentoRelevante():
    """Obtém tabela com movimentos que foram classificados como relevantes para a solução proposta

    Returns:
        pandas.DataFrame: Tabela com movimentos relevantes
    """

   # Constroi tabela auxiliar de movimentos
    tabMovimentoRelevante = pd.read_csv('dados/01_tabelas_auxiliares/tabMovimentoRelevante.csv',
                                        sep=';', usecols=['cod_movimento', 'relevante', 'resolucao_merito'])

    # Força Id da tabela para float
    tabMovimentoRelevante['cod_movimento'] = tabMovimentoRelevante['cod_movimento'].astype(
        np.float)

    # Filtra apenas os movimentos relevants
    tabMovimentoRelevante = tabMovimentoRelevante[tabMovimentoRelevante.relevante == True]
    print('tabMovimentosRelevantes carregada')
    print(tabMovimentoRelevante.shape)
    
    return tabMovimentoRelevante


In [13]:
def melhorar_qualidade_movimento(diretorio):
    """Melhora a qualidade dos movimentos de acordo com a necessidade da solução proposta

    Args:
        diretorio (string): Local onde o arquivo processos_full.csv se encontra

    Returns:
        pandas.DataFrame: tabela de movimentos criada
    """
    
    # Carga dos movimentos
    print('Carregando os dados: ' + local_dados + diretorio + 'processos_full.csv')
    tabela = pd.read_csv(local_dados + diretorio + 'processos_full.csv',
                         usecols=['dadosBasicos_numero', 'cod_movimento', 'descricao_movimento', 'descricao_movimento_pai',
                                  'data_movimento', 'data_movimento_str', 'dadosBasicos_assunto_principal'],
                         low_memory=False)
    print('Dados originais')
    print(tabela.shape)

    # Filtrar por assunto_principal
    tabela = tabela[tabela.dadosBasicos_assunto_principal == True]
    tabela.pop('dadosBasicos_assunto_principal')
    print('Dados filtrados por assunto principal')
    print(tabela.shape)

    # Remove data_movimento não preenchida
    tabela.dropna(subset=['data_movimento'], inplace=True)
    # Converte colunas para data
    tabela['data_movimento'] = pd.to_datetime(tabela['data_movimento'])
    # Remover todos os movimentos posteriores ao dia de hoje... pois não fazem sentido
    tabela = tabela[tabela.data_movimento <= datetime.now()]
    print('Dados removidas data_movimento inconsistentes')
    print(tabela.shape)

    tabela.drop_duplicates(keep='first', inplace=True)
    print('Dados removidas duplicações')
    print(tabela.shape)
    
    return tabela


In [14]:
def movimentos_relevantes(diretorio, tabMovimentoRelevante):
    """Chama procedimento de melhoria da qualidade dos movimentos, filtra movimentos relevantes e salva o resultado

    Args:
        diretorio (string): Local onde o arquivo processos_full.csv se encontra
        tabMovimentoRelevante (pandas.DataFrame): tabela com movimentos relevantes selecionados
    """
    
    tabela = melhorar_qualidade_movimento(diretorio)

    # Movimentações Relevantes
    tabela = pd.merge(tabela, tabMovimentoRelevante, on='cod_movimento')
    tabela.drop(['relevante'], axis=1, inplace=True)
    print('Tabela apenas com movimentos relevantes')
    print(tabela.shape)

    # Criar arquivo .CSV
    print('Salvando: ' + local_dados + diretorio + 'tab_processos_full.csv')
    tabela.to_csv(local_dados + diretorio +
                  'tab_movimentos_relevantes.csv', index=False)
    print()

In [15]:
# Fluxo principal de execução

print("Criação tabMovimentosRelevantes iniciada")

tabMovimentoRelevante = obter_tabMovimentoRelevante()

movimentos_relevantes('00_justica_federal/processos-trf1/', tabMovimentoRelevante)
movimentos_relevantes('00_justica_federal/processos-trf2/', tabMovimentoRelevante)
movimentos_relevantes('00_justica_federal/processos-trf3/', tabMovimentoRelevante)
movimentos_relevantes('00_justica_federal/processos-trf4/', tabMovimentoRelevante)
movimentos_relevantes('00_justica_federal/processos-trf5/', tabMovimentoRelevante)
print("Finalizada criação tabMovimentosRelevantes!!")



Criação tabMovimentosRelevantes iniciada
tabMovimentosRelevantes carregada
(85, 3)
Carregando os dados: dados/02_tabelas_transformadas/00_justica_federal/processos-trf1/processos_full.csv
Dados originais
(3018472, 7)
Dados filtrados por assunto principal
(2577467, 6)
Dados removidas data_movimento inconsistentes
(2577463, 6)
Dados removidas duplicações
(2468063, 6)
Tabela apenas com movimentos relevantes
(15114, 7)
Salvando: dados/02_tabelas_transformadas/00_justica_federal/processos-trf1/tab_processos_full.csv

Carregando os dados: dados/02_tabelas_transformadas/00_justica_federal/processos-trf2/processos_full.csv
Dados originais
(118416, 7)
Dados filtrados por assunto principal
(117806, 6)
Dados removidas data_movimento inconsistentes
(117806, 6)
Dados removidas duplicações
(117805, 6)
Tabela apenas com movimentos relevantes
(10694, 7)
Salvando: dados/02_tabelas_transformadas/00_justica_federal/processos-trf2/tab_processos_full.csv

Carregando os dados: dados/02_tabelas_transformadas